Построение рекомендательной системы. Соревнование на Kaggle https://www.kaggle.com/c/sbermarket-internship-competition/overview

В качестве тренировочных данных представляется датасет с историей заказов 20000 пользователей вплоть до даты отсечки, которая разделяет тренировочные и тестовые данные по времени.

train.csv:
- user_id - уникальный id пользователя
- order_completed_at - дата заказа
- cart - список уникальных категорий (category_id), из которых состоял заказ

В качестве прогноза необходимо для каждой пары пользователь-категория из примера сабмита вернуть 1, если категория будет присутствовать в следующем заказе пользователя, или 0 в ином случае. Список категорий для каждого пользователя в примере сабмита - это все категории, которые он когда-либо заказывал.



# Первичный анализ
В соревновании говорится о создании рекомендательной системы для покупателей СберМаркета. Как известно, суть создания рекомандательной системы состоит в том, чтобы по известному рейтингу user-item предложить клиенту товар, который он купит с наибольшей вероятностью, и главной задачей является предсказание рейтинга для тех пар user-item, для которых он изначально неизвестен. Однако в данной задаче, как сказано в описании, в тестовой выборке для каждого покупателя присутствуют только те категории, которые он до этого покупал, то есть те, которые присутствуют в тестовой выборке. Таким образом нам известны предпочтения для всех пар user-item, которые могут нас интересовать, значит главной нашей задачей является правильное определение рейтинга исходя из ранее сделанных покупок для всех пар user-item.

Первой идеей, которая приходит в голову, является определение рейтинга просто как доли заказов клиента, в которых была куплена конкретная категория. Так мы получим первичную оценку вероятности покупки клиентом категории вне зависимости от того, когда был сделан заказ. Давайте посчитаем эту долю, но перед этим проведем небольшой разведочный анализ

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
df['order_completed_at'] = pd.to_datetime(df.order_completed_at)

In [4]:
df.groupby(['user_id', 'order_completed_at']).count()

# заказы и количество категорий в них для каждого id

cart
user_id order_completed_at       
0       2020-07-19 09:59:17     8
        2020-08-24 08:55:32    25
        2020-09-02 07:38:25    11
1       2019-05-08 16:09:41     1
        2020-01-17 14:44:23     6
...                           ...
19998   2020-09-01 08:12:32     7
        2020-09-02 15:03:23     4
19999   2020-08-31 18:54:24     1
        2020-08-31 19:32:08     1
        2020-08-31 20:08:24     1

[209406 rows x 1 columns]

In [5]:
sorted(df.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart'].unique())

# Список количества заказов в выборке

[3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 78,
 79,
 80,
 81,
 83,
 85,
 86,
 87,
 88,
 90,
 91,
 92,
 93,
 94,
 95,
 97,
 98,
 99,
 101,
 104,
 107,
 108,
 111,
 113,
 114,
 115,
 116,
 119,
 120,
 124,
 125,
 126,
 127,
 133,
 134,
 137,
 145,
 154,
 155,
 163,
 165,
 187,
 213]

Найдем индексы тех людей, которые есть в сэмпле и дальнейший анализ будем делать для них, так как на данном этапе мы предполагаем, что с помощью рейтингов похожих клиентов удастся найти рейтинг определенного клиента, поэтому нас не интересуют id, которых нет в сэмпле.

In [6]:
id_ind = list(set(sample['id'].apply(lambda x: int(x.split(';')[0])).values))

# id которые есть в сэмпле

In [7]:
cat_ind = list(set(sample['id'].apply(lambda x: int(x.split(';')[1])).values))

# индексы категорий которые есть в сэмпле

In [8]:
df_ind = df.query('user_id in @id_ind')

# нужные нам индексы наблюдений

# Простая доля
Теперь посчитаем долю заказов, в которой присутствовала категория

In [9]:
order_num = df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count().values.reshape(13036,)
order_num

# количество заказов для id

array([3, 9, 7, ..., 3, 3, 3], dtype=int64)

In [10]:
cat_num = df_ind.groupby(['user_id', 'cart']).count()
cat_num

# в скольких заказах была категория

order_completed_at
user_id cart                    
0       5                      1
        10                     1
        14                     2
        20                     1
        22                     1
...                          ...
19998   398                    2
        409                    1
        415                    2
        420                    2
        798                    1

[790449 rows x 1 columns]

In [11]:
div = []
for i, j in zip(order_num, np.array(cat_num.groupby('user_id').count()['order_completed_at'])):
    div.append((str(i)+' ') * j)
div = np.array(list(map(int, ' '.join(div).split())))
share = np.array(cat_num).reshape(790449,) / div
share_df = df_ind.groupby(['user_id', 'cart']).count().reset_index().drop('order_completed_at', axis=1)
share_df['share'] = share

# Посчитаем долю заказов

In [12]:
share_df

,user_id,cart,share
0,0,5,0.333333
1,0,10,0.333333
2,0,14,0.666667
3,0,20,0.333333
4,0,22,0.333333
...,...,...,...
790444,19998,398,0.666667
790445,19998,409,0.333333
790446,19998,415,0.666667
790447,19998,420,0.666667


In [13]:
sample['merge'] = list(zip(sample['id'].apply(lambda x: int(x.split(';')[0])).values, sample['id'].apply(lambda x: int(x.split(';')[1])).values))
share_df['merge'] = list(zip(share_df['user_id'].values, share_df['cart'].values))

# Создадим столбцы для объединения таблиц

In [14]:
result = sample.merge(share_df, on='merge')[['id', 'share']]

In [15]:
result

,id,share
0,0;133,0.333333
1,0;5,0.333333
2,0;10,0.333333
3,0;396,0.333333
4,0;14,0.666667
...,...,...
790444,19998;26,0.333333
790445,19998;31,0.333333
790446,19998;29,0.333333
790447,19998;798,0.333333


In [16]:
result['target'] = 0
result.loc[result[result['share'] >= 0.5].index , 'target'] = 1

# Предскажем покупку для всех категорий, доля которых больше 0.5

In [17]:
result[['id', 'target']].to_csv('submission1.csv', index=False)

# Анализ временной структуры
Следующей мыслью, которая приходит в голову является анализ того, как зависит вероятность покупки от более поздних и более ранних заказов. Логично предположить, что заказы, которые делались недавно больше влияют на новый заказ, чем более старые, так как у человека со временем могут меняться предпочтения, любимые категории и так далее. Чтобы проверить эту гипотезу посчитаем среднюю схожесть между первым и последним заказами, а также между последним и предпоследним, затем сравним их и сделаем соответствующие выводы. Схожесть будем считать как долю одинаковых категорий в заказах

In [18]:
orders = df_ind.groupby(['user_id', 'order_completed_at']).count()
orders

cart
user_id order_completed_at       
0       2020-07-19 09:59:17     8
        2020-08-24 08:55:32    25
        2020-09-02 07:38:25    11
1       2019-05-08 16:09:41     1
        2020-01-17 14:44:23     6
...                           ...
19997   2020-08-31 11:04:05     1
        2020-08-31 11:48:23    17
19998   2020-08-30 12:15:55     8
        2020-09-01 08:12:32     7
        2020-09-02 15:03:23     4

[160557 rows x 1 columns]

In [19]:
sim_first = np.array([])
random_ids = np.random.choice(id_ind, size=100, replace=False)

for i in random_ids:
    first = orders.loc[i].index[0]
    last = orders.loc[i].index[-1]
    first_set = set(df_ind[(df_ind['user_id'] == i) & (df_ind['order_completed_at'] == first)]['cart'])
    last_set = set(df_ind[(df_ind['user_id'] == i) & (df_ind['order_completed_at'] == last)]['cart'])
    union = sorted(list(first_set | last_set))
    first_list = [0 for x in range(len(union))]
    last_list = [0 for x in range(len(union))]
    for i in first_set:
        first_list[union.index(i)] = 1
    for j in last_set:
        last_list[union.index(j)] = 1
    sim_first = np.append(sim_first, sum(np.array(first_list) + np.array(last_list) == 2) / len(first_list))
    
# Схожесть между первым и последним заказами для 100 случайных id

In [20]:
sim_second = np.array([])

for i in random_ids:
    penult = orders.loc[i].index[-2]
    last = orders.loc[i].index[-1]
    penult_set = set(df_ind[(df_ind['user_id'] == i) & (df_ind['order_completed_at'] == penult)]['cart'])
    last_set = set(df_ind[(df_ind['user_id'] == i) & (df_ind['order_completed_at'] == last)]['cart'])
    union = sorted(list(penult_set | last_set))
    penult_list = [0 for x in range(len(union))]
    last_list = [0 for x in range(len(union))]
    for i in penult_set:
        penult_list[union.index(i)] = 1
    for j in last_set:
        last_list[union.index(j)] = 1
    sim_second = np.append(sim_second, sum(np.array(penult_list) + np.array(last_list) == 2) / len(penult_list))
    
# Схожесть между предпоследним и последним заказами для 100 случайных id

In [21]:
print('Схожесть между заказами:\nПервым и последним = {:.3f}\nПердпоследним и последним = {:.3f}'.format(np.mean(sim_first), np.mean(sim_second)))

Схожесть между заказами:
Первым и последним = 0.170
Пердпоследним и последним = 0.223


# Модификация доли
Действительно похожесть между последним и предпоследним заказами выше, чем между последним и первым. Стоит это учесть при расчете доли категории в заказах, так как последние заказы явно должны иметь больший вклад, чем более ранние.
Для начала просто посчитаем долю по последним трем заказам и сделаем предсказание 1 для тех категорий, которые были куплены 2 или 3 раза

In [22]:
a = df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']
for i in sorted(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart'].unique()):
    print('Количество заказов: {}, id с таким количеством заказов: {} штук'.format(i, sum(a == i)))

Количество заказов: 3, id с таким количеством заказов: 1888 штук
Количество заказов: 4, id с таким количеством заказов: 1465 штук
Количество заказов: 5, id с таким количеством заказов: 1195 штук
Количество заказов: 6, id с таким количеством заказов: 973 штук
Количество заказов: 7, id с таким количеством заказов: 817 штук
Количество заказов: 8, id с таким количеством заказов: 742 штук
Количество заказов: 9, id с таким количеством заказов: 611 штук
Количество заказов: 10, id с таким количеством заказов: 490 штук
Количество заказов: 11, id с таким количеством заказов: 412 штук
Количество заказов: 12, id с таким количеством заказов: 383 штук
Количество заказов: 13, id с таким количеством заказов: 344 штук
Количество заказов: 14, id с таким количеством заказов: 305 штук
Количество заказов: 15, id с таким количеством заказов: 306 штук
Количество заказов: 16, id с таким количеством заказов: 279 штук
Количество заказов: 17, id с таким количеством заказов: 218 штук
Количество заказов: 18, id с 

In [23]:
df_ind['last_1'] = 0    # последний заказ
df_ind['last_2'] = 0    # предпоследний заказ
df_ind['last_3'] = 0    # предпредпоследний заказ

df_ind['user_id_time'] = list(zip(df_ind['user_id'], df_ind['order_completed_at']))

ind1 = np.array(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 1
val_id1 = list(orders.iloc[ind1].reset_index()['user_id'])
val_time1 = list(orders.iloc[ind1].reset_index()['order_completed_at'])
val_id_time1 = list(zip(val_id1, val_time1))
df_ind.loc[df_ind.query('user_id_time in @val_id_time1').index, 'last_1'] = 1
last_1 = df_ind.groupby(['user_id', 'cart']).sum()['last_1'].values
share_df['last_1'] = last_1

ind2 = np.array(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 2
val_id2 = list(orders.iloc[ind2].reset_index()['user_id'])
val_time2 = list(orders.iloc[ind2].reset_index()['order_completed_at'])
val_id_time2 = list(zip(val_id2, val_time2))
df_ind.loc[df_ind.query('user_id_time in @val_id_time2').index, 'last_2'] = 1
last_2 = df_ind.groupby(['user_id', 'cart']).sum()['last_2'].values
share_df['last_2'] = last_2

ind3 = np.array(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 3
val_id3 = list(orders.iloc[ind3].reset_index()['user_id'])
val_time3 = list(orders.iloc[ind3].reset_index()['order_completed_at'])
val_id_time3 = list(zip(val_id3, val_time3))
df_ind.loc[df_ind.query('user_id_time in @val_id_time3').index, 'last_3'] = 1
last_3 = df_ind.groupby(['user_id', 'cart']).sum()['last_3'].values
share_df['last_3'] = last_3

# Посчитаем бинарные признаки, была ли категория в последних трех заказах

C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [24]:
share_df['target'] = 0
share_df.loc[share_df.query('last_1 + last_2 + last_3 in [2, 3]').index, 'target'] = 1
share_df

# Посчитаем новый таргет

,user_id,cart,share,merge,last_1,last_2,last_3,target
0,0,5,0.333333,"(0, 5)",0,1,0,0
1,0,10,0.333333,"(0, 10)",0,1,0,0
2,0,14,0.666667,"(0, 14)",0,1,1,1
3,0,20,0.333333,"(0, 20)",0,0,1,0
4,0,22,0.333333,"(0, 22)",0,1,0,0
...,...,...,...,...,...,...,...,...
790444,19998,398,0.666667,"(19998, 398)",0,1,1,1
790445,19998,409,0.333333,"(19998, 409)",1,0,0,0
790446,19998,415,0.666667,"(19998, 415)",0,1,1,1
790447,19998,420,0.666667,"(19998, 420)",0,1,1,1


In [25]:
result2 = sample.merge(share_df, on='merge').rename(columns={'target_y': 'target'})[['id', 'target']]

In [26]:
result2[['id', 'target']].to_csv('submission2.csv', index=False)

В финансах есть идея, что лучшее предсказание цены актива, это его цена в данный момент. Давайте воспользуемся этим и просто предскажем категории в следующей покупке теми категориями, которые были куплены в последний раз

In [27]:
result3 = sample.merge(share_df, on='merge').rename(columns={'last_1': 'target'})[['id', 'target']]

In [28]:
result3[['id', 'target']].to_csv('submission3.csv', index=False)

Как уже было установлено, схожесть заказов со временем убывает, поэтому найдем среднее взвешенное последних трех заказов. Возьмем вес 0.5 для последнего заказа, 0.3 для предпоследнего и 0.2 для предпредпоследнего и предскажем 1 для среднего взвешенного >= 0.5. Таким образом предсказание будет аналогичным обычной доле из последних трех заказов, но для всех категорий, которые были в последнем заказе предскажется единица. Получится некоторое объединение предыдущих двух решений

In [29]:
share_df['target_w'] = 0
share_df.loc[share_df.query('last_1*0.5 + last_2*0.3 + last_3*0.2 >= 0.5').index, 'target_w'] = 1
share_df

,user_id,cart,share,merge,last_1,last_2,last_3,target,target_w
0,0,5,0.333333,"(0, 5)",0,1,0,0,0
1,0,10,0.333333,"(0, 10)",0,1,0,0,0
2,0,14,0.666667,"(0, 14)",0,1,1,1,1
3,0,20,0.333333,"(0, 20)",0,0,1,0,0
4,0,22,0.333333,"(0, 22)",0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
790444,19998,398,0.666667,"(19998, 398)",0,1,1,1,1
790445,19998,409,0.333333,"(19998, 409)",1,0,0,0,1
790446,19998,415,0.666667,"(19998, 415)",0,1,1,1,1
790447,19998,420,0.666667,"(19998, 420)",0,1,1,1,1


In [30]:
result4 = sample.merge(share_df, on='merge').rename(columns={'target_w': 'target'})[['id', 'target']]

In [31]:
result4[['id', 'target']].to_csv('submission4.csv', index=False)

Теперь предскажем 1 для всех категорий с долей больше 0.5, и которые были в последних 3-х заказах

In [32]:
share_df['target_1'] = 0
share_df.loc[share_df.query('share >= 0.5').index, 'target_1'] = 1
share_df.loc[share_df.query('last_1 + last_2 + last_3 == 3').index, 'target_1'] = 1
share_df

,user_id,cart,share,merge,last_1,last_2,last_3,target,target_w,target_1
0,0,5,0.333333,"(0, 5)",0,1,0,0,0,0
1,0,10,0.333333,"(0, 10)",0,1,0,0,0,0
2,0,14,0.666667,"(0, 14)",0,1,1,1,1,1
3,0,20,0.333333,"(0, 20)",0,0,1,0,0,0
4,0,22,0.333333,"(0, 22)",0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
790444,19998,398,0.666667,"(19998, 398)",0,1,1,1,1,1
790445,19998,409,0.333333,"(19998, 409)",1,0,0,0,1,0
790446,19998,415,0.666667,"(19998, 415)",0,1,1,1,1,1
790447,19998,420,0.666667,"(19998, 420)",0,1,1,1,1,1


In [33]:
result5 = sample.merge(share_df, on='merge').rename(columns={'target_1': 'target'})[['id', 'target']]

In [34]:
result5[['id', 'target']].to_csv('submission5.csv', index=False)

Усилим наши предположения и теперь предскажем 1 для всех категорий с долей больше 0.5, и которые были в последних 2-х заказах

In [35]:
share_df['target_2'] = 0
share_df.loc[share_df.query('share >= 0.5').index, 'target_2'] = 1
share_df.loc[share_df.query('last_1 + last_2 == 2').index, 'target_2'] = 1
share_df

,user_id,cart,share,merge,last_1,last_2,last_3,target,target_w,target_1,target_2
0,0,5,0.333333,"(0, 5)",0,1,0,0,0,0,0
1,0,10,0.333333,"(0, 10)",0,1,0,0,0,0,0
2,0,14,0.666667,"(0, 14)",0,1,1,1,1,1,1
3,0,20,0.333333,"(0, 20)",0,0,1,0,0,0,0
4,0,22,0.333333,"(0, 22)",0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
790444,19998,398,0.666667,"(19998, 398)",0,1,1,1,1,1,1
790445,19998,409,0.333333,"(19998, 409)",1,0,0,0,1,0,0
790446,19998,415,0.666667,"(19998, 415)",0,1,1,1,1,1,1
790447,19998,420,0.666667,"(19998, 420)",0,1,1,1,1,1,1


In [36]:
result6 = sample.merge(share_df, on='merge').rename(columns={'target_2': 'target'})[['id', 'target']]

In [37]:
result6[['id', 'target']].to_csv('submission6.csv', index=False)

Теперь предскажем 1 для всех категорий, у которых доля во всех заказах и в последних трех заказах больше 0.5

In [38]:
share_df['target_3'] = 0
share_df.loc[share_df.query('share >= 0.5').index, 'target_3'] = 1
share_df.loc[share_df.query('last_1 + last_2 + last_3 in [2, 3]').index, 'target_3'] = 1

share_df

,user_id,cart,share,merge,last_1,last_2,last_3,target,target_w,target_1,target_2,target_5
0,0,5,0.333333,"(0, 5)",0,1,0,0,0,0,0,0
1,0,10,0.333333,"(0, 10)",0,1,0,0,0,0,0,0
2,0,14,0.666667,"(0, 14)",0,1,1,1,1,1,1,1
3,0,20,0.333333,"(0, 20)",0,0,1,0,0,0,0,0
4,0,22,0.333333,"(0, 22)",0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
790444,19998,398,0.666667,"(19998, 398)",0,1,1,1,1,1,1,1
790445,19998,409,0.333333,"(19998, 409)",1,0,0,0,1,0,0,0
790446,19998,415,0.666667,"(19998, 415)",0,1,1,1,1,1,1,1
790447,19998,420,0.666667,"(19998, 420)",0,1,1,1,1,1,1,1


In [39]:
result7 = sample.merge(share_df, on='merge').rename(columns={'target_3': 'target'})[['id', 'target']]

In [40]:
result7[['id', 'target']].to_csv('submission7.csv', index=False)

# Строим модель

Будем использовать градиентный бустинг. Но для начала нужно извлечь как можно больше признаков для каждой пары user-item. Возьмем такие признаки:

1) Доля заказов клиента, в которых присутствовала категория

2) Бинарный признак - принимает значение 1, если категория была в последнем заказе

3) Аналогично 2-му признаку, но для предпоследнего заказа

4) Популярность категории - доля заказов всех клиентов, в которых присутствовала категория

Также у нас уть информация об id и номер категории, которые тоже содержат в себе ценную информацию. One hot encoding в данном случае нежелателен, так как слишком сильно раздует наше признаковое пространство, и большинство признаков будут крайн разреженными, поэтому применим mean target encoding к переменным user_id и cart.

5) user_id

6) cart

Для обучения возьмем всю выборку, кроме последнего заказа - информация о покупке для него будет зависимой переменной

In [67]:
df['y'] = 0
df['user_id_time'] = list(zip(df['user_id'], df['order_completed_at']))
last_ind = np.array(df.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 1
orders2 = df.groupby(['user_id', 'order_completed_at']).count()
last_val_time = list(orders2.iloc[last_ind].reset_index()['order_completed_at'])
last_val_id = list(orders2.iloc[last_ind].reset_index()['user_id'])
last_val_id_time = list(zip(last_val_id, last_val_time))
df.loc[df.query('user_id_time in @last_val_id_time').index, 'y'] = 1
df_model = df.drop(df.query('user_id_time in @last_val_id_time').index)    # удалили из выборки последний заказ для каждого id
data = df_model.groupby(['user_id', 'cart']).count().reset_index().drop('order_completed_at', axis=1)    # таблица для обучающих данных
y = df.groupby(['user_id', 'cart']).sum().loc[df_model.groupby(['user_id', 'cart']).count().index]['y'].values
df_model.drop('y', axis=1, inplace=True)
data.drop('user_id_time', axis=1, inplace=True)

data['y'] = y

# Посчитаем зависимую переменную

In [68]:
order_num2 = df_model.drop('user_id_time', axis=1).groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart'].values.reshape(20000,)
cat_num2 = df_model.drop('user_id_time', axis=1).groupby(['user_id', 'cart']).count()

div = []
for i, j in zip(order_num2, np.array(cat_num2.groupby('user_id').count()['order_completed_at'])):
    div.append((str(i)+' ') * j)
div = np.array(list(map(int, ' '.join(div).split())))
share = np.array(cat_num2).reshape(1031269 ,) / div
data['share'] = share

# Посчитаем долю заказов

In [69]:
df_model['last'] = 0
df_model['penult'] = 0

orders3 = df_model.groupby(['user_id', 'order_completed_at']).count()

last_ind2 = np.array(df_model.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 1
last_val_id2 = list(orders3.iloc[last_ind2].reset_index()['user_id'])
last_val_time2 = list(orders3.iloc[last_ind2].reset_index()['order_completed_at'])
last_val_id_time2 = list(zip(last_val_id2, last_val_time2))
df_model.loc[df_model.query('user_id_time in @last_val_id_time2').index, 'last'] = 1
last = df_model.groupby(['user_id', 'cart']).sum()['last'].values
data['last'] = last

penult_ind2 = np.array(df_model.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 2
penult_val_id2 = list(orders3.iloc[penult_ind2].reset_index()['user_id'])
penult_val_time2 = list(orders3.iloc[penult_ind2].reset_index()['order_completed_at'])
penult_val_id_time2 = list(zip(penult_val_id2, penult_val_time2))
df_model.loc[df_model.query('user_id_time in @penult_val_id_time2').index, 'penult'] = 1
penult = df_model.groupby(['user_id', 'cart']).sum()['penult'].values
data['penult'] = penult

# Посчитаем бинарные признаки, была ли категория в последнем и предпоследнем заказах

In [70]:
pop_cat = df_model.groupby('cart').count()['user_id'] / orders2.groupby('user_id').count()['cart'].sum()
data_temp = data.sort_values('cart')

pop = []
for i, j in zip(pop_cat, np.array(data.sort_values('cart').groupby('cart').count()['user_id'])):
    pop.append((str(i)+' ') * j)
pop = np.array(list(map(float, ' '.join(pop).split())))
data_temp['popularity'] = pop
data = data_temp.loc[data.index]

# Посчитаем популярность категорий

In [71]:
from category_encoders.target_encoder import TargetEncoder
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['user_id', 'cart', 'share', 'last', 'penult', 'popularity']],data['y'],
                                                    test_size=0.2, random_state=1, stratify = data['y'])

te1 = TargetEncoder(smoothing=1)
X_train['user_id_te'] = te1.fit_transform(X_train['user_id'].astype(str), y_train)
X_test['user_id_te'] = te1.transform(X_test['user_id'].astype(str))

te2 = TargetEncoder(smoothing=1)
X_train['cart_te'] = te2.fit_transform(X_train['cart'].astype(str), y_train)
X_test['cart_te'] = te2.transform(X_test['cart'].astype(str))

# Преобразовываем user_id и cart

Далее выделим признаки для сэмпла

In [72]:
data_sample = df_ind.groupby(['user_id', 'cart']).count().reset_index().drop('order_completed_at', axis=1)
# таблица для предсказания

data_sample['share'] = share_df['share']
# ранее посчитанная доля категории в заказах

In [73]:
df_ind['last'] = 0
df_ind['penult'] = 0
df_ind['user_id_time'] = list(zip(df_ind['user_id'], df_ind['order_completed_at']))
orders = df_ind.groupby(['user_id', 'order_completed_at']).count()

last_ind3 = np.array(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 1
last_val_id3 = list(orders.iloc[last_ind3].reset_index()['user_id'])
last_val_time3 = list(orders.iloc[last_ind3].reset_index()['order_completed_at'])
last_val_id_time3 = list(zip(last_val_id3, last_val_time3))
df_ind.loc[df_ind.query('user_id_time in @last_val_id_time3').index, 'last'] = 1
last = df_ind.groupby(['user_id', 'cart']).sum()['last'].values
data_sample['last'] = last

penult_ind3 = np.array(df_ind.groupby(['user_id', 'order_completed_at']).count().groupby('user_id').count()['cart']).cumsum() - 2
penult_val_id3 = list(orders.iloc[penult_ind3].reset_index()['user_id'])
penult_val_time3 = list(orders.iloc[penult_ind3].reset_index()['order_completed_at'])
penult_val_id_time3 = list(zip(penult_val_id3, penult_val_time3))
df_ind.loc[df_ind.query('user_id_time in @penult_val_id_time3').index, 'penult'] = 1
penult = df_ind.groupby(['user_id', 'cart']).sum()['penult'].values
data_sample['penult'] = penult

# Посчитаем бинарные признаки, была ли категория в последнем и предпоследнем заказах

C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\solmi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [74]:
pop_cat = df_ind.groupby('cart').count()['user_id'] / orders2.groupby('user_id').count()['cart'].sum()
data_sample_temp = data_sample.sort_values('cart')

pop = []
for i, j in zip(pop_cat, np.array(data_sample.sort_values('cart').groupby('cart').count()['user_id'])):
    pop.append((str(i)+' ') * j)
pop = np.array(list(map(float, ' '.join(pop).split())))
data_sample_temp['popularity'] = pop
data_sample = data_sample_temp.loc[data_sample.index]

# Посчитаем популярность категорий

In [75]:
te1 = TargetEncoder(smoothing=1)
data['user_id_te'] = te1.fit_transform(data['user_id'].astype(str), data['y'])
data_sample['user_id_te'] = te1.transform(data_sample['user_id'].astype(str))

te2 = TargetEncoder(smoothing=1)
data['cart_te'] = te2.fit_transform(data['cart'].astype(str), data['y'])
data_sample['cart_te'] = te2.transform(data_sample['cart'].astype(str))

# Преобразовываем user_id и cart

Признаки выделены, теперь обучим модель

In [77]:
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

n_estimators = np.append(np.array(list(range(40, 121, 40))), np.array([500, 1000]))
num_leaves = np.array(list(range(10, 71, 10)))

searcher = GridSearchCV(lgb.LGBMClassifier(objective="binary"),
                        [{'n_estimators' : n_estimators, 'num_leaves' : num_leaves}],
                            scoring = 'f1', cv = 5)
searcher.fit(X_train, y_train)
y_pred = searcher.predict(X_test)
    
print("n_estimators: {}, num_leaves: {}, valid F1 score: {:.4f}".format(searcher.best_params_['n_estimators'], searcher.best_params_['num_leaves'], f1_score(y_test, y_pred)))

reg_alpha: 8.858667904100823, valid F1 score: 0.4040


In [80]:
from sklearn.model_selection import cross_val_score
cross_val_score(lgb.LGBMClassifier(objective="binary", n_estimators=searcher.best_params_['n_estimators'], num_leaves = searcher.best_params_['num_leaves']), data[['user_id_te', 'cart_te', 'share', 'last', 'penult', 'popularity']], data['y'], cv=5, scoring='f1')

array([0.37488643, 0.39263397, 0.40924531, 0.45170095, 0.48964472])

In [81]:
data_sample['merge'] = list(zip(data_sample['user_id'], data_sample['cart']))
data_sample = sample.merge(data_sample, on='merge')

In [83]:
booster = lgb.LGBMClassifier(objective="binary", n_estimators=searcher.best_params_['n_estimators'], num_leaves = searcher.best_params_['num_leaves'])
booster.fit(data[['user_id_te', 'cart_te', 'share', 'last', 'penult', 'popularity']], data['y'])
target = booster.predict(data_sample[['user_id_te', 'cart_te', 'share', 'last', 'penult', 'popularity']])

In [84]:
sample['target'] = target

In [85]:
sample[['id', 'target']].to_csv('submission9.csv', index=False)